In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/d3_wikihow.csv")

In [3]:
len(df[df.Source == "wikihow"])

15914

In [4]:
df1 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-01.txt.gz", error_bad_lines=False, sep="\t")
df2 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-02.txt.gz", error_bad_lines=False, sep="\t")
df3 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-03.txt.gz", error_bad_lines=False, sep="\t")
df4 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-04.txt.gz", error_bad_lines=False, sep="\t")
df5 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-05.txt.gz", error_bad_lines=False, sep="\t")
df6 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-06.txt.gz", error_bad_lines=False, sep="\t")
df7 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-07.txt.gz", error_bad_lines=False, sep="\t")
df8 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-08.txt.gz", error_bad_lines=False, sep="\t")
df9 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-09.txt.gz", error_bad_lines=False, sep="\t")
df10 = pd.read_csv("data/AOL-user-ct-collection/user-ct-test-collection-10.txt.gz", error_bad_lines=False, sep="\t")
aol = df1.append([df2,df3,df4,df5,df6,df7,df8,df9,df10], ignore_index=True)

/Users/jarana/anaconda3/envs/keras/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# Preprocessing
aol = aol.drop_duplicates(['AnonID', 'Query'], keep='first')
aol = aol[~aol['Query'].str.lower().str.contains('|'.join(["www.", ".com", ".uk", ".us", ".edu"]), na=False)]
aol["QueryTime"] = pd.to_datetime(aol["QueryTime"])
aol = aol.dropna(subset=['Query'])

In [6]:
howto = aol[aol['Query'].str.lower().str.contains("how to", na=False)]

In [11]:
from keras.preprocessing.text import Tokenizer

MAX_NUM_WORDS = 100000

corpus = howto.Query.tolist() + df.Query.tolist()

# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(corpus)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


Found 32630 unique tokens.


In [32]:
MAX_SEQUENCE_LENGTH = max([len(i) for i in corpus])
x = pad_sequences(tokenizer.texts_to_sequences(corpus), maxlen=MAX_SEQUENCE_LENGTH)

In [17]:
from utils import *

max_words = len(word_index)
path = "/Users/jarana/workspace/WikiHow-Task-Based/"
embedding_layer = get_pretrain_embeddings(path, max_words, word_index)


Indexing word vectors.
Found 1917494 word vectors.
Preparing embedding matrix.


W0907 17:06:31.219952 4510414272 deprecation_wrapper.py:119] From /Users/jarana/anaconda3/envs/keras/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



Token num: 32630, Found Tokens: 28931


In [34]:
from keras.models import Sequential
from keras.layers import Dense, Activation, GlobalMaxPooling1D

model = Sequential([embedding_layer, GlobalMaxPooling1D()])

In [35]:
model.predict(x).shape

(175464, 300)

In [37]:
howto

,AnonID,Query,QueryTime,ItemRank,ClickURL
754,2722,how to make tulle butterflies,2006-04-12 15:05:40,3,http://www.butterflybreeders.com
7904,13373,how to paint concrete floor,2006-04-21 10:11:57,1,http://interiordec.about.com
9253,15565,pageant hair extentions how tos,2006-04-10 02:28:50,NaN,NaN
11969,26111,how to uninstall a kitchen facuit,2006-05-18 19:15:38,NaN,NaN
11970,26111,how to uninstall a kitchen faucet,2006-05-18 19:15:48,NaN,NaN
12508,29859,how to change timing belts,2006-03-18 09:28:38,1,http://www.miata.net
12509,29859,how to change hyndia timing belts,2006-03-18 09:30:27,NaN,NaN
12832,29894,how to get free web on nextel phones,2006-05-24 19:41:01,NaN,NaN
14037,30011,how to make quajada,2006-05-21 12:32:46,1,http://www.geocities.com
14071,30011,how to relief gas,2006-05-22 21:15:42,1,http://www.americarx.com
